<a href="https://colab.research.google.com/github/JayantJharkhande3000/CS6910_Assignment_03_EtoH/blob/main/Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb --upgrade

     |████████████████████████████████| 1.8 MB 6.7 MB/s 
     |████████████████████████████████| 181 kB 70.3 MB/s 
     |████████████████████████████████| 145 kB 56.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=965595f092c1ed8996db10d2c2e6095d163ae4081060f2934dc22608ce0bc197
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
wandb.init(project="final_A2", entity="jharkhandejayant")

wandb: Currently logged in as: jharkhandejayant. Use `wandb login --relogin` to force relogin


In [5]:
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,LSTM,GRU,SimpleRNN,Input,Dropout,TimeDistributed,RepeatVector,dot,BatchNormalization,concatenate,multiply,Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [6]:
train_data_path = "/content/drive/MyDrive/DATA/lexicons/hi.translit.sampled.train.tsv"
test_data_path = "/content/drive/MyDrive/DATA/lexicons/hi.translit.sampled.test.tsv"
validation_data_path = "/content/drive/MyDrive/DATA/lexicons/hi.translit.sampled.dev.tsv"

In [7]:
# creating array for input (English) and Target for Output words
English_texts = []
hindi_texts = []
english_alphabets = set()
hindi_alphabets = set()
# Calling the training data 
with open(train_data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
# printing training data
print("\n Printing Raw Hindi and English DataSet \n")
for i in range(400,410):
  print(lines[i])


# Deleting the last line in training data
del lines[-1]

# separating the english and hindi words as input text and target text
for line in lines:
    hindi_text, english_text, _ = line.split("\t")
    english_text = "\t"+english_text + "\n"
    hindi_text = "\t"+hindi_text + "\n"

    English_texts.append(english_text)
    hindi_texts.append(hindi_text)

    for character in english_text:
        if character not in english_alphabets:
            english_alphabets.add(character)
    for character in hindi_text:
        if character not in hindi_alphabets:
            hindi_alphabets.add(character)

english_alphabets.add(" ")
hindi_alphabets.add(" ")

# total number of sample 
print("\n")
print("Total number of English input samples in training :", len(English_texts))
print("Total number of  Hindi output samples in training :", len(hindi_texts))


# characters in hindi and english vocabulary
english_alphabets = sorted(list(english_alphabets))
print("\n english_alphabets English  \n")
print(english_alphabets)
hindi_alphabets = sorted(list(hindi_alphabets))
print("\n hindi_alphabets Hindi  \n")
print(hindi_alphabets)

english_tokens = len(english_alphabets)
print("\nTotal number of characters in English  dictionary :",english_tokens)

hindi_tokens = len(hindi_alphabets)
print("\nTotal number of characters in Hindi dictionary :", hindi_tokens)

english_max_length = max([len(txt) for txt in English_texts])
print("Max sequence length in the english :", english_max_length)

hindi_max_length = max([len(txt) for txt in hindi_texts])
print("Max sequence length in the hindi :", hindi_max_length)

# converting character of hindi and english dictionary to numbers

eng_alp_to_num = dict([(char, i) for i, char in enumerate(english_alphabets)])
print("\n english alphabets converted to numeric value \n", eng_alp_to_num)
hin_alp_to_num = dict([(char, i) for i, char in enumerate(hindi_alphabets)])
print("\n hindi alphabets converted to numeric value \n", hin_alp_to_num)



 Printing Raw Hindi and English DataSet 

अचार	achaar	2
अचार	achar	2
अचिन	achin	3
अचूक	achook	1
अचूक	achuk	2
अचेत	achet	2
अचेत	ahet	1
अचेतन	achetan	3
अच्छा	accha	1
अच्छा	achcha	2


Total number of English input samples in training : 44204
Total number of  Hindi output samples in training : 44204

 english_alphabets English  

['\t', '\n', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 hindi_alphabets Hindi  

['\t', '\n', ' ', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ']

Total number of characters in English  dictionary : 29

Total number of characters in Hindi dictionary : 66
Max sequence length in the english : 22
Max sequence l

In [8]:
# English_To_Hindi
class English_To_Hindi:
    # Initializing the parameters 
    def __init__(self,cell_type,embedding_size,hidden_layer_size,encoder_layers_size,decoder_layers_size,dropouts,epochs,batch_size,
                 english_alphabets,hindi_alphabets,english_max_length,hindi_max_length,english_tokens,hindi_tokens,
                 eng_alp_to_num,hin_alp_to_num,activation,optimizer,train_data_path,test_data_path,validation_data_path ):
        self.cell_type = cell_type
        self.embedding_size = embedding_size
        self.hidden_layer_size = hidden_layer_size
        self.encoder_layers_size = encoder_layers_size
        self.decoder_layers_size = decoder_layers_size
        self.dropouts=dropouts
        self.epochs=epochs
        self.batch_size=batch_size
        self.english_alphabets = english_alphabets
        self.hindi_alphabets = hindi_alphabets 
        self.english_max_length = english_max_length
        self.hindi_max_length = hindi_max_length
        self.english_tokens =  english_tokens
        self.hindi_tokens =  hindi_tokens
        self.eng_alp_to_num =  eng_alp_to_num
        self.hin_alp_to_num =  hin_alp_to_num
        self.activation =  activation
        self.optimizer =  optimizer
        self.train_data_path =  train_data_path
        self.test_data_path =  test_data_path
        self.validation_data_path = validation_data_path
        
    # Converting english and hindi text to numeric array
    def text_to_array(self,path):
        d = pd.read_csv(path,sep="\t",header=None,error_bad_lines=False)
        d = d.dropna()

        decoder_target_data = np.zeros((d.shape[0],self.hindi_max_length,self.hindi_tokens), dtype="float32")

        for i,target_text in enumerate(d[0]):
            target_text = '\t'+target_text+'\n'
            for t, char in enumerate(target_text):
                if t > 0:
                    decoder_target_data[i, t - 1, self.hin_alp_to_num[char]] = 1.0
            decoder_target_data[i, t:, self.hin_alp_to_num["\n"]] = 1.0

        return ([[self.eng_alp_to_num[letter] for letter in list('\t'+word+'\n')] for word in d[1]]),\
                ([[self.hin_alp_to_num[letter] for letter in list('\t'+word+'\n')] for word in d[0]]),decoder_target_data
    
    # Encoder Decoder function for train, test, validation set
    def encoder_decoder(self):
        # Dict for index to char
        self.inv_input_token_index = dict({(value,key) for key,value in self.eng_alp_to_num.items()})
        self.inv_target_token_index = dict({(value,key) for key,value in self.hin_alp_to_num.items()})

        train_eng_input_encoder,train_eng_out_decoder,self.train_hin_target_decoder = self.text_to_array(self.train_data_path)
        vali_eng_input_encoder,vali_eng_out_decoder,self.vali_hin_target_decoder = self.text_to_array(self.validation_data_path)
        test_eng_input_encoder,test_eng_out_decoder,self.test_hin_target_decoder = self.text_to_array(self.test_data_path)

        self.train_eng_input_encoder = sequence.pad_sequences(train_eng_input_encoder,maxlen=self.english_max_length,padding="post")
        self.train_eng_out_decoder = sequence.pad_sequences(train_eng_out_decoder,maxlen=self.hindi_max_length,padding="post")
        self.vali_eng_input_encoder = sequence.pad_sequences(vali_eng_input_encoder,maxlen=self.english_max_length,padding="post")
        self.vali_eng_out_decoder = sequence.pad_sequences(vali_eng_out_decoder,maxlen=self.hindi_max_length,padding="post")
        self.test_eng_input_encoder = sequence.pad_sequences(test_eng_input_encoder,maxlen=self.english_max_length,padding="post")
        self.test_eng_out_decoder = sequence.pad_sequences(test_eng_out_decoder,maxlen=self.hindi_max_length,padding="post")
        


    def build_model(self):
        # Encoder box 
        encoder_inputs = Input(shape=(None,))
        x = Embedding(self.english_tokens,self.embedding_size,input_length=self.english_max_length)(encoder_inputs)
        for _ in range(self.encoder_layers_size):
            # if cell type = LSTM
            if self.cell_type == "lstm":
                x,state_h,state_c = LSTM(self.hidden_layer_size,return_state=True,return_sequences=True,dropout=self.dropouts)(x)
                encoder_states = [state_h,state_c]
            # if cell type = RNN
            elif self.cell_type == "rnn":
                x,state_c = SimpleRNN(self.hidden_layer_size,return_state=True,return_sequences=True,dropout=self.dropouts)(x)
                encoder_states = [state_c]
            # if cell type = GRU
            elif self.cell_type == "gru":
                x,state_c = GRU(self.hidden_layer_size,return_state=True,return_sequences=True,dropout=self.dropouts)(x)
                encoder_states = [state_c]
                        
        # Decoder box 
        decoder_inputs = Input(shape=(None,))
        decoder_embb = Embedding(self.hindi_tokens,self.embedding_size,input_length=self.hindi_max_length)(decoder_inputs)
        # if cell type = LSTM
        if self.cell_type == "lstm":
            decoder_lstm,state_h,state_c = LSTM(self.hidden_layer_size, return_sequences=True,
                                                return_state=True,dropout=self.dropouts)(decoder_embb,initial_state=encoder_states)
        # if cell type = RNN
        elif self.cell_type == "rnn":
            decoder_lstm,state_h = SimpleRNN(self.hidden_layer_size, return_sequences=True,
                                             return_state=True,dropout=self.dropouts)(decoder_embb,initial_state=encoder_states)
        # if cell type = GRU
        elif self.cell_type == "gru":
            decoder_lstm,state_h = GRU(self.hidden_layer_size, return_sequences=True,
                                       return_state=True,dropout=self.dropouts)(decoder_embb,initial_state=encoder_states)

        for i in range(self.decoder_layers_size-1):
            # if cell type = LSTM
            if self.cell_type == "lstm":
                decoder_lstm,state_h,state_c = LSTM(self.hidden_layer_size, return_sequences=True,return_state=True,dropout=self.dropouts)(decoder_lstm)
            # if cell type = RNN
            elif self.cell_type == "rnn":
                decoder_lstm,state_h = SimpleRNN(self.hidden_layer_size, return_sequences=True,return_state=True,dropout=self.dropouts)(decoder_lstm)
            # if cell type = GRU
            elif self.cell_type == "gru":
                decoder_lstm,state_h = GRU(self.hidden_layer_size, return_sequences=True,return_state=True,dropout=self.dropouts)(decoder_lstm)
        

        decoder_outputs = TimeDistributed(Dense(self.hindi_tokens, activation= self.activation))(decoder_lstm)

        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

        return model


    def run(self,model):
        # Compile & run training
        model.compile(optimizer= self.optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

        model.fit([self.train_eng_input_encoder,self.train_eng_out_decoder], self.train_hin_target_decoder,
                self.batch_size,
                self.epochs,
                validation_data=([self.vali_eng_input_encoder,self.vali_eng_out_decoder], self.vali_hin_target_decoder),
                callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5),
                                            wandb.keras.WandbCallback()])                                   

In [9]:
sweep_config={
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters':{
        'epochs':{
            'values':[3,5]
        },
        'embedding_size':{
            'values':[8,12,16,20]
        },
        'encoder_layers_size':{
            'values':[1,2,3]
        },
        'decoder_layers_size':{
            'values':[1,2,3]
        },
        'cell_type':{
            'values':["rnn","gru","lstm"]
        },
        'dropouts':{
            'values':[0,0.2,0.3,0.4]
        },
        'hidden_layer_size':{
            'values':[16,32,64,128,256]
        },
        'batch_size':{
            'values':[32,64]
        },
        'activation':{
            'values': ["softmax"]
        }, 
        'optimizer':{
            'values':["Adam"]
        } 
    }
}

In [10]:
sweep_id = wandb.sweep(sweep_config, project="final_A2", entity="jharkhandejayant")


Create sweep with ID: ez9bkk5v
Sweep URL: https://wandb.ai/jharkhandejayant/final_A2/sweeps/ez9bkk5v


In [11]:
def train():
    config_defaults={
        'cell_type':"lstm",
        'embedding_size':16,
        'hidden_layer_size': 128,
        'encoder_layers_size':2,
        'decoder_layers_size':2,
        'dropouts':0,
        'epochs':5,
        'batch_size': 64,
        'activation': "softmax",
        'optimizer':"Adam" 
        }
    
    wandb.init(config=config_defaults)
    config=wandb.config
    English_To_Hindi_translation = English_To_Hindi(config.cell_type,config.embedding_size,config.hidden_layer_size,config.encoder_layers_size,
                                                    config.decoder_layers_size,config.dropouts,config.epochs,config.batch_size,
                                                    english_alphabets,hindi_alphabets,english_max_length, hindi_max_length,english_tokens,
                                                    hindi_tokens,eng_alp_to_num,hin_alp_to_num, config.activation,config.optimizer,
                                                    train_data_path,test_data_path,validation_data_path )
    English_To_Hindi_translation.encoder_decoder()
    English_To_Hindi_translation_Model = English_To_Hindi_translation.build_model()
    English_To_Hindi_translation.run(English_To_Hindi_translation_Model)

In [12]:
wandb.agent(sweep_id,train)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 3gc0cd9h with config:
wandb: 	activation: softmax
wandb: 	batch_size: 64
wandb: 	cell_type: rnn
wandb: 	decoder_layers_size: 2
wandb: 	dropouts: 0.3
wandb: 	embedding_size: 12
wandb: 	encoder_layers_size: 1
wandb: 	epochs: 3
wandb: 	hidden_layer_size: 32
wandb: 	optimizer: Adam


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/3
691/691 [==============================] - 18s 22ms/step - loss: 1.2121 - accuracy: 0.7191 - val_loss: 0.9373 - val_accuracy: 0.7498 - _timestamp: 1652619393.0000 - _runtime: 25.0000
Epoch 2/3
691/691 [==============================] - 16s 23ms/step - loss: 0.9666 - accuracy: 0.7407 - val_loss: 0.9030 - val_accuracy: 0.7518 - _timestamp: 1652619409.0000 - _runtime: 41.0000
Epoch 3/3
691/691 [==============================] - 15s 21ms/step - loss: 0.9422 - accuracy: 0.7437 - val_loss: 0.8787 - val_accuracy: 0.7587 - _timestamp: 1652619423.0000 - _runtime: 55.0000


accuracy,▁▇█
epoch,▁▅█
loss,█▂▁
val_accuracy,▁▃█
val_loss,█▄▁
accuracy,0.74374
best_epoch,2
best_val_loss,0.87871
epoch,2
loss,0.94218
val_accuracy,0.75875


wandb: Agent Starting Run: hrjhox04 with config:
wandb: 	activation: softmax
wandb: 	batch_size: 64
wandb: 	cell_type: gru
wandb: 	decoder_layers_size: 3
wandb: 	dropouts: 0.3
wandb: 	embedding_size: 16
wandb: 	encoder_layers_size: 3
wandb: 	epochs: 3
wandb: 	hidden_layer_size: 16
wandb: 	optimizer: Adam


...

Epoch 1/3
691/691 [==============================] - 24s 28ms/step - loss: 1.4508 - accuracy: 0.7098 - val_loss: 1.0347 - val_accuracy: 0.7408 - _timestamp: 1652626388.0000 - _runtime: 31.0000
Epoch 2/3
691/691 [==============================] - 18s 27ms/step - loss: 1.0152 - accuracy: 0.7385 - val_loss: 0.9269 - val_accuracy: 0.7538 - _timestamp: 1652626406.0000 - _runtime: 49.0000
Epoch 3/3
691/691 [==============================] - 18s 26ms/step - loss: 0.9471 - accuracy: 0.7467 - val_loss: 0.8947 - val_accuracy: 0.7589 - _timestamp: 1652626424.0000 - _runtime: 67.0000


accuracy,▁▆█
epoch,▁▅█
loss,█▂▁
val_accuracy,▁▆█
val_loss,█▃▁
accuracy,0.74674
best_epoch,2
best_val_loss,0.89472
epoch,2
loss,0.94709
val_accuracy,0.75894


wandb: Agent Starting Run: 9i5f0azt with config:
wandb: 	activation: softmax
wandb: 	batch_size: 64
wandb: 	cell_type: gru
wandb: 	decoder_layers_size: 2
wandb: 	dropouts: 0.2
wandb: 	embedding_size: 16
wandb: 	encoder_layers_size: 3
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 64
wandb: 	optimizer: Adam


Epoch 1/5
691/691 [==============================] - 69s 86ms/step - loss: 1.1764 - accuracy: 0.7236 - val_loss: 0.9243 - val_accuracy: 0.7519 - _timestamp: 1652626511.0000 - _runtime: 77.0000
Epoch 2/5
277/691 [===========>..................] - ETA: 33s - loss: 0.9520 - accuracy: 0.7432

wandb: Ctrl + C detected. Stopping sweep.
